In [1]:
import os
import json
from rdflib import Literal, URIRef, Dataset
from rdflib.namespace import RDF, FOAF, RDFS, SKOS, Namespace

folder_path = "/Users/simonsnopko/Desktop/Škola/bakalarska praca/DrugBank"

DRUGBANK = Namespace("http://go.drugbank.com/drugs/")
BIOREGISTRY = Namespace("https://bioregistry.io/drugbank:")

json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

specific_keys = ["drugbank_id", "name", "description", "synonyms"]

ds = Dataset()

graph_uri = URIRef("http://drugbank_ontology.org")

g = ds.graph(graph_uri)
g.bind("drugbank", DRUGBANK)
g.bind("bioregistry", BIOREGISTRY)
g.bind("skos", SKOS)

for json_file in json_files:
    with open(os.path.join(folder_path, json_file)) as f:
        json_dict = json.load(f)
        for key in json_dict.keys():
            if key in specific_keys:
                json_uri = URIRef(BIOREGISTRY + json_file[:-5])
                if key == "drugbank_id":
                    key_uri = SKOS.notation
                    xref_uri = Literal("drugbank:" + json_dict[key])
                    g.add((json_uri, key_uri, xref_uri))
                elif key == "name":
                    key_uri = SKOS.prefLabel
                    g.add((json_uri, key_uri, Literal(json_dict[key], lang='en')))
                elif key == "description":
                    if json_dict[key] != None:
                        key_uri = SKOS.definition
                        g.add((json_uri, key_uri, Literal(json_dict[key])))
                elif key == "synonyms":
                    for synonym in json_dict[key]:
                        if synonym["language"] == "english":
                            key_uri = SKOS.altLabel
                            g.add((json_uri, key_uri, Literal(synonym["synonym"], lang='en')))

ds.serialize(destination="drugbank created ontology.nq", format="nquads")


<Graph identifier=N6fa7d082bafc4673b9f24d0e713bf616 (<class 'rdflib.graph.Dataset'>)>